In [39]:
pip install praw

In [40]:
import praw
import pandas as pd
# Reddit credentials, password stored in .env 
# PRAW setup and login goes here, omitted for privacy
reddit = praw.Reddit(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, password=PASSWORD, user_agent=USER_AGENT, username=USERNAME)

In [41]:
# Set up some sort of corpus of keywords to snag specific reddit entries
all_locs = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
all_locs = pd.json_normalize(all_locs['data'])
all_locs = all_locs.drop(columns=['edit_at','id'])
def cleanlinks(json):
    links_out = []
    for link in json:
        links_out.append(link['url'])
    return links_out
all_locs['links'] = all_locs['links'].apply(cleanlinks)
all_locs['date'] = pd.to_datetime(all_locs['date'],format='%Y-%m-%d')
all_tags = all_locs['tags'].copy()
tags = set()
for taglist in all_tags:
  for tag in taglist:
    if tag not in tags:
      tags.add(tag)

print(tags)

{'homeless', '', 'gun', 'body-cam', 'racial-profiling', 'knee', 'pregnant', 'inhumane-treatment', 'spray', 'celebrity', 'hide-badge', 'headlock', 'lgbtq+', 'person-with-disability', 'foam-bullet', 'tackle', 'bike', 'conceal', 'non-protest', 'live-round', 'lrad', 'shield', 'shoot', 'shove', 'zip-tie', 'dog', 'kick', 'gas', 'taser', 'horse', 'choke', 'push', 'projectile', 'knee-on-neck', 'marking-round', 'arrest', 'vehicle', 'medic', 'punch', 'elderly', 'politician', 'sexual-assault', 'baton', 'child', 'strike', 'bean-bag', 'abuse-of-power', 'grab', 'throw', 'protester', 'death', 'drive', 'pepper-ball', 'pepper-spray', 'rubber-bullet', 'threaten', 'tase', 'incitement', 'bystander', 'beat', 'mace', 'explosive', 'tear-gas-canister', 'journalist', 'wooden-bullet', 'less-lethal', 'stun-grenade', 'legal-observer', 'property-destruction', 'tear-gas'}


In [42]:
all_locs.head(20)

,links,state,city,description,tags,geolocation,name,date,date_text
0,[https://www.youtube.com/watch?v=s7MM1VauRHo],Washington,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st
1,[https://mobile.twitter.com/chadloder/status/1...,Washington,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th
2,[https://twitter.com/gunduzbaba1905/status/126...,Washington,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st
3,[https://www.reddit.com/r/Bad_Cop_No_Donut/com...,Washington,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st
4,[https://www.fox10phoenix.com/news/video-shows...,Washington,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st
5,[https://www.reddit.com/r/Seattle/comments/gu3...,Washington,Seattle,A sheriff throws a canister of tear gas into a...,"[less-lethal, protester, tear-gas, tear-gas-ca...","47.6117535, -122.3363867",Police throw tear gas at peaceful protesters,2020-05-31,May 31st
6,[https://twitter.com/EDDIFUL/status/1267338642...,Washington,Seattle,An officer on a bike pulls down a protester wa...,"[protester, tackle]",,Police officer pulls protester to the ground u...,2020-05-31,May 31st
7,[https://twitter.com/The_Stepover/status/12672...,Washington,Seattle,"In this video, officers wrestle with two prote...","[protester, punch, tackle]",,Law enforcement officer punches pinned protester,2020-05-31,May 31st
8,[https://twitter.com/The_Stepover/status/12672...,Washington,Seattle,Police are standing in a row holding bikes and...,"[less-lethal, protester, stun-grenade, tear-gas]",,Law enforcement officers throw flashbangs into...,2020-05-31,(Believed to be) May 31st
9,[https://www.reddit.com/r/Seattle/comments/gv0...,Washington,Seattle,Police pepper spray peacefully protesting crowd.,"[less-lethal, pepper-spray, protester]",,Police pepper spray crowd,2020-06-01,June 1st


In [43]:
# Discard irrelevant tags

tags.discard('')
tags.discard('medic')
tags.discard('bike')
tags.discard('pregnant')
tags.discard('legal-observer')
tags.discard('politician')
tags.discard('incitement')
tags.discard('homeless')
tags.discard('elderly')
tags.discard('vehicle')
tags.discard('inhumane-treatment')
tags.discard('journalist')
tags.discard('throw')
tags.discard('threaten')
tags.discard('horse')
tags.discard('child')
tags.discard('shield')
tags.discard('dog')
tags.discard('conceal')
tags.discard('bystander')
tags.discard('drive')
tags.discard('person-with-disability')
tags.discard('property destruction')
tags.discard('celebrity')


newtags = set()
for tag in tags:
  tag = newtags.add(tag.replace('-',' '))
tags = newtags
print(tags)

{'gun', 'knee', 'spray', 'stun grenade', 'headlock', 'lgbtq+', 'tear gas', 'sexual assault', 'live round', 'tackle', 'rubber bullet', 'lrad', 'shoot', 'shove', 'kick', 'gas', 'taser', 'zip tie', 'choke', 'push', 'projectile', 'wooden bullet', 'arrest', 'property destruction', 'punch', 'less lethal', 'knee on neck', 'pepper ball', 'baton', 'pepper spray', 'foam bullet', 'strike', 'body cam', 'grab', 'protester', 'death', 'non protest', 'tase', 'bean bag', 'beat', 'hide badge', 'mace', 'explosive', 'abuse of power', 'racial profiling', 'tear gas canister', 'marking round'}


In [44]:
# Create categories of tags to reduce overlapping categories

tags_list = list(tags)

for i in range(len(tags_list)):
    if tags_list[i] == 'tackle' or tags_list[i] == 'push' or tags_list[i] == 'shove' or tags_list[i] == 'strike' or tags_list[i] == 'grab' or tags_list[i] == 'beat' or tags_list[i] == 'punch' or tags_list[i] == 'choke' or tags_list[i] == 'headlock' or tags_list[i] == 'kick' or tags_list[i] == 'knee on neck' or tags_list[i] == 'knee':
        tags_list[i] = 'physical violence'
    elif tags_list[i] == 'foam bullet' or tags_list[i] == 'bullet' or tags_list[i] == 'wooden bullet' or tags_list[i] == 'rubber bullet' or tags_list[i] == 'live round' or tags_list[i] == 'marking round':
        tags_list[i] = 'bullet'
    elif tags_list[i] == 'tear gas' or tags_list[i] == 'projectile' or tags_list[i] == 'pepper spray' or tags_list[i] == 'zip tie' or tags_list[i] == 'gas' or tags_list[i] == 'pepper ball' or tags_list[i] == 'stun grenade' or tags_list[i] == 'lrad' or tags_list[i] == 'taser' or tags_list[i] == 'baton' or tags_list[i] == 'tase' or tags_list[i] == 'mace' or tags_list[i] == 'spray' or tags_list[i] == 'bean bag' or tags_list[i] == 'tear gas canister':
        tags_list[i] = 'non-lethal weapon'
    elif tags_list[i] == 'lgbtq+' or tags_list[i] == 'racial profiling' or tags_list[i] == 'profiling':
        tags_list[i] = 'profiling'
  
tags_list = set(tags_list)

print(tags_list) 


{'gun', 'less lethal', 'protester', 'death', 'bullet', 'non protest', 'shoot', 'body cam', 'profiling', 'hide badge', 'explosive', 'abuse of power', 'sexual assault', 'physical violence', 'non-lethal weapon', 'arrest', 'property destruction'}


In [45]:
# Grabbing 1000 hottest posts on Reddit at the moment. Will filter for police use of force later

data = []

# Other possible subreddits: publicfreakout, allcopnodonut
for submission in reddit.subreddit("news").hot(limit=1000):
  data.append([submission.id, submission.title, submission.score, submission.subreddit, submission.url, 
               submission.num_comments, submission.selftext, submission.created])

# We'll need a way to get coordinates for a given post, before we include that in df
col_names = ['id', 'title', 'score', 'subreddit', 'url', 
             'num_comments', 'text', 'created']
df = pd.DataFrame(data, columns=col_names)

df.head()

,id,title,score,subreddit,url,num_comments,text,created
0,j5m8qf,President Trump and US Government COVID-19 Meg...,1523,news,https://www.reddit.com/r/news/comments/j5m8qf/...,8728,**This thread is for discussing all things rel...,1.601943e+09
1,j6r0jh,Family of Tyson employee in Iowa who died of C...,54471,news,https://www.desmoinesregister.com/story/money/...,1634,,1.602107e+09
2,j6s6aj,Federal appeals court rules Trump can't block ...,6441,news,https://www.cnn.com/2020/10/07/politics/trump-...,284,,1.602111e+09
3,j6np78,Death of Georgia teen shot by police ruled a h...,12063,news,https://abcnews.go.com/US/death-georgia-teen-s...,432,,1.602091e+09
4,j6qekn,Exclusive: OxyContin maker Purdue nears guilty...,3976,news,https://www.reuters.com/article/us-purdue-phar...,209,,1.602105e+09


In [46]:
import requests
from bs4 import BeautifulSoup
import re

# Get the url of the reddit post
for url in df['url']:
  # Get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # Get tags from metadata for the site
  sitetags = set()
  for meta in soup.find_all('meta'):
    if meta is None:
      continue
    meta = str(meta)
    meta = meta.lower()
    meta = re.sub('[\W_]+',' ', meta)
    for tag in str(meta).split():
      sitetags.add(tag)
  tags_final = tags_list & sitetags
  if tags_final:
    # Some matches found, print title and matched tags
    print(soup.title.text.strip(), tags_final)

Tyson Foods sued over Columbus Junction worker's COVID-19 death {'death'}
Death of Georgia teen shot by police ruled a homicide by medical examiner - ABC News {'death'}
Tap water now safe to drink in Texas city where boy died from brain-eating amoeba - CBS News {'death'}
Florida man arrested for threatening census taker with gun {'gun'}
US Postal Service worker arrested, accused of dumping mail, including ballots sent to New Jersey residents - ABC7 Los Angeles {'arrest'}
St. Louis couple indicted for waving guns at protesters {'gun'}
Police accused of beating porcupines to death with batons - ABC News {'death'}
Lake County father arrested after 12-year-old son fatally shoots himself, deputies say {'gun'}
Ex-cop charged in Floyd's death freed on $1 million bond {'death'}
Attorney for Amber Guyger retained by officer who shot Jonathan Price - ABC News {'death'}
Lawsuit pursued after child was hit while getting off school bus; parents angry driver wasn't arrested {'arrest'}
Father, 2 chil

In [47]:
# Test grabbing the contents of the articles themselves to imporve spaCy NLP
# Get the url of the reddit post
import re

for url in df['url'][:10]:
  # Get the HTML from the url
  try:
    r = requests.get(url, timeout=10)
  except:
    continue
  soup = BeautifulSoup(r.text)
  # Get text from website
  output_text = " ".join([x.text for x in soup.find_all('p')])
  output_text = re.sub('[^a-zA-Z0-9.,\']+', ' ', output_text).strip()
  print(output_text)

Sorry, for some reason reddit can't be reached.
The children of a Tyson Foods meatpacker in Columbus Junction who died of COVID 19 have sued the company over what they say was a lack of protective measures for its workers. Pedro Cano's family said he worked on the eastern Iowa pork processing plant's line in late March and April without a mask or gloves, according to a lawsuit filed in Johnson County District Court on Sept. 23. They also allege Cano worked elbow to elbow with other employees and did not receive guidance from his bosses about the threat of COVID 19. Cano began to feel symptoms of the virus on April 2, four days before the plant was closed because of rapidly spreading infections, according to the lawsuit. At the time, Tyson reported that 24 employees had tested positive for the virus, a figure that eventually ballooned to 522, according to the Iowa Occupational Safety and Health Administration. Cano's family alleges gross negligence and fraudulent misrepresentation by Ty